In [1]:
import pandas as pd 
import numpy as np
import gc
import os
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim import corpora
import warnings
warnings.filterwarnings("ignore")

In [67]:
work_path = os.open('...',os.O_RDONLY)
files = os.listdir(work_path)
files= sorted(files)
files = files[1:]
start_bid = files.index('...')
end_bid = files.index('...')
files = files[start_bid:end_bid]
files.remove('...')

In [3]:
# some parameters
time_step = 7
threshold_view_amount = 500

In [4]:
file_path = '...'
url_dict = np.load('url_dict.npy')
uuid_dict = np.load('id_dict.npy')
df_dict = url_dict.item()
model = Word2Vec.load('url_model')
uuid_dict = uuid_dict.item()

In [6]:
temp_csv = pd.read_csv(file_path + '...', sep = '\t', header = None, compression='gzip')

In [7]:
temp_csv = temp_csv.iloc[:, [2,4]]
temp_csv = temp_csv.dropna()
temp_csv[4] = temp_csv[4].map(lambda x: df_dict[x])
temp_csv[4] = temp_csv[4].map(lambda x: str(x))

In [9]:
temp_csv.shape

(4940189, 2)

In [10]:
len(temp_csv.groupby(2).apply(lambda x: ' '.join(list(x[4]))))

41684

In [20]:
temp_csv[2] = temp_csv[2].map(lambda x: uuid_dict[x])

In [71]:
def map_uuid2vuid(uuid):
    try:
        vuid = uuid_dict[uuid]
    except:
        vuid = 'nan'
    
    return vuid

In [86]:
def preprocess4corpus(file_path,file_name): 
    temp_csv = pd.read_csv(file_path + file_name, sep = '\t', header = None, compression='gzip')
    temp_csv = temp_csv.iloc[:, [2,4]]
    temp_csv = temp_csv.dropna()
    temp_csv[4] = temp_csv[4].map(lambda x: df_dict[x])
    temp_csv[4] = temp_csv[4].map(lambda x: str(x))
    temp_csv[2] = temp_csv[2].map(lambda x: map_uuid2vuid(x))
    try:
        temp_csv = temp_csv.loc[temp_csv[2] != 'nan']
    except:
        temp_csv = temp_csv
    df_session = temp_csv.groupby(2).apply(lambda x: ' '.join(list(x[4])))
    df_session = pd.DataFrame(df_session)
    df_session = df_session.reset_index()
    df_session.columns = ['id', 'url_session']
    df_session.url_session = df_session.url_session.map(lambda x: x.split(' '))
    df_session['session_length'] = df_session.url_session.map(lambda x: len(x))
    df_session = df_session.loc[df_session.session_length<=threshold_view_amount]
    
    return df_session

def calcu_user2vec_single_day(raw_des):
    user_corpus = dictionary.doc2bow(raw_des)
    uservec = np.zeros([1,200], dtype = float)
    for j in range(len(user_corpus)):
        try:
            uservec += tfidf[user_corpus][j][1]*model[dictionary.get(user_corpus[j][0])]
        except:
            uservec += np.zeros([1,200], dtype = float)
    return uservec

In [ ]:
for file in files:
    ssuser_vec_dict = {}
    temp_csv = preprocess4corpus(file_path, file)
    
    corpus = temp_csv.url_session.values
    dictionary = corpora.Dictionary(corpus)
    dictionary.filter_extremes(no_below=1, no_above=1, keep_n=300000, keep_tokens=None)
    corpus_dict = [dictionary.doc2bow(text) for text in corpus]
    tfidf = TfidfModel(corpus_dict)
    
    temp_csv['user_vec_per_day'] = temp_csv.url_session.map(lambda x: calcu_user2vec_single_day(x))

    vuid_list = temp_csv.uuid.values
    vuid_vec = temp_csv.user_vec_per_day.values
    vuvec_dict = {}
    
    for i in range(len(vuid_list)):
        vuvec_dict[vuid_list[i]] = vuid_vec[i]
    np.save('user_vec_per_day/user_vec'+ file[-15:-7] +'.npy', vuvec_dict)
    print(file + '   end!')